Fundamentals of Data Analysis Project 2020

Student : Michelle Moran 

Task details : 


Perform and explain simple linear regression using Python on the powerproduction dataset

Goal : Accurately predict wind turbine output from wind speed values using the data set as a basis.
Using GIT repository  for the submission including the following items : 

1. Jupyter notebook that performs simple linear regression on the data set
2. An explanation of your regression and an analysis of the accuracy
3. Standard items in a git repository such as a README
4. Compare a simple linear regression to other types of regression on this data set


References : 


